In [14]:
import requests
import random

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "EEVE-Korean-10.8B"

# 주제별 제시어 목록
TOPICS = {
    "동물": ["기린", "코끼리", "펭귄", "늑대", "하마", "토끼", "여우", "호랑이", "부엉이", "사자"],
    "과일": ["사과", "바나나", "수박", "포도", "딸기", "복숭아", "파인애플", "키위", "오렌지", "자두"],
    "직업": ["의사", "경찰", "소방관", "선생님", "요리사", "프로그래머", "운동선수", "가수", "배우", "디자이너"],
    "교통수단": ["자전거", "기차", "비행기", "버스", "자동차", "오토바이", "지하철", "택시", "트럭", "요트"],
    "음식": ["김치", "떡볶이", "치킨", "비빔밥", "라면", "초밥", "불고기", "스파게티", "피자", "햄버거"]
}

LIAR_GAME_SYSTEM_PROMPT = """
당신은 '라이어 게임(Liar Game)'에 참가한 플레이어입니다. 다음 규칙과 역할 지침을 철저히 따르세요:

🎮 [게임 개요]
- 게임은 총 3~8명의 플레이어로 진행되며, 각 플레이어는 'Player N' 형식의 이름을 가집니다.
- 모든 플레이어는 ‘일반’ 또는 ‘라이어’ 역할 중 하나를 가집니다. 이 역할은 비공개이며, 당신만 자신의 역할을 압니다.
- 일반 플레이어는 '제시어(keyword)'를 알고 있고, 이를 기반으로 애매하고 자연스러운 힌트를 제시해야 합니다.
- 라이어는 제시어를 모릅니다. 하지만 들키지 않기 위해 마치 알고 있는 것처럼 힌트를 줘야 합니다.

🎲 [발언 순서]
- 발언 순서는 게임 시작 시 무작위로 정해지며, 모든 라운드에서 동일하게 유지됩니다.

🗣️ [힌트 라운드 규칙]
- 게임은 총 3라운드이며, 각 라운드에서 모든 플레이어는 한 문장으로 힌트를 제시합니다.
- 힌트는 제시어를 직접 언급하거나 노골적으로 암시하면 안 됩니다.
- 힌트는 너무 어색하거나 뜬금없으면 안 됩니다. 가능한 한 자연스럽고 사람처럼 행동하세요.

🕵️ [추측 라운드 규칙]
- 힌트 라운드 이후, 모든 플레이어는 다른 플레이어들의 힌트를 보고 라이어가 누구일지 추측합니다.
- 누가 수상한지, 누가 일반 플레이어처럼 보이는지 짧은 코멘트를 남기세요.
- 존재하지 않는 플레이어나 라운드를 언급하지 마세요.
- 다른 플레이어가 사람이거나 AI인지 구분할 수 없습니다. 오직 'Player N' 형식의 이름으로만 판단하세요.
- 추측은 자연스럽고 사람처럼 말해야 하며, 절대 단정짓지 말고 추측에 기반하세요.

🎭 [예시]
힌트 예시:
- "목이 정말 길죠."
- "조용한 성격인 것 같아요."
- "다리가 은근히 눈에 띄네요."

※ 힌트는 모두 한 문장이어야 하며, 너무 구체적이거나 너무 일반적이지 않아야 합니다.
※ 제시어를 직접 언급하거나 뚜렷한 연상어는 피하세요.
※ 자연스럽고 짧게 말하는 것이 중요합니다.

추측 멘트 예시:
- "Player 3은 힌트가 좀 무난했어. 일반일 수도 있을 것 같아."
- "Player 2는 말이 너무 조심스러웠어. 수상해."
"""

def get_ai_response(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": LIAR_GAME_SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        return result.get("message", {}).get("content", "").strip()
    except Exception as e:
        return f"❌ Error: {e}"

def run_game(total_players: int, human_players: int = 1):
    topic = random.choice(list(TOPICS.keys()))
    keyword = random.choice(TOPICS[topic])

    print(f"\n📌 이번 주제: {topic}\n")

    ai_players = total_players - human_players
    player_names = [f"Player {i+1}" for i in range(total_players)]
    roles = ["Human"] * human_players + ["AI"] * ai_players
    combined = list(zip(player_names, roles))
    random.shuffle(combined)
    player_names, roles = zip(*combined)
    player_names, roles = list(player_names), list(roles)

    liar_index = random.randint(0, total_players - 1)
    liar_player = player_names[liar_index]
    print(f"🕵️‍♂️ [시크릿] 라이어는 👉 {liar_player} ({roles[liar_index]})\n")

    for round_num in range(1, 4):
        print(f"\n🎯 ===== 라운드 {round_num} 시작 ===== 🎯")
        if round_num == 1:
            player_order = player_names.copy()
            random.shuffle(player_order)
        print("🎲 고정 힌트 발표 순서:")
        for i, name in enumerate(player_order, 1):
            print(f"{i}. {name}")
        print("=" * 60)

        hints = {}
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]
            print(f"\n🎙️ {player}의 힌트:")

            print(f"📌 당신의 역할: {'라이어' if idx == liar_index else '일반 플레이어'}")
            print(f"📌 주제: {topic}")

            if role == "Human":
                if idx == liar_index:
                    print("⚠️ 당신은 라이어입니다. 제시어는 모릅니다.")
                    hint = input("👉 한 문장으로 애매한 힌트를 입력하세요: ")
                else:
                    print(f"✅ 당신은 일반 플레이어입니다. 제시어는 '{keyword}'입니다.")
                    hint = input("👉 한 문장으로 힌트를 입력하세요: ")
            else:
                if idx == liar_index:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 당신은 라이어입니다. 제시어는 비공개입니다. 주제는 '{topic}'입니다. 한 문장으로 힌트를 주세요."
                else:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 제시어는 '{keyword}'입니다. 당신은 일반 플레이어입니다. 제시어를 직접 언급하지 말고 한 문장으로 힌트를 주세요."
                hint = get_ai_response(prompt)

            hints[player] = hint
            print(f"💬 힌트: {hint}")
            print("=" * 60)

        print("\n💭 추측 라운드 시작!")
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]

            print(f"\n🕵️ {player}의 의심 발언:")

            if role == "Human":
                print("💡 참고 힌트:")
                for other, hint in hints.items():
                    if other != player:
                        print(f"- {other}: {hint}")
                guess = input("👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요: ")
            else:
                summary = "\n".join([f"{name}: {hints[name]}" for name in player_order if name != player])
                if idx == liar_index:
                    prompt = f"당신은 라이어입니다. 주제는 '{topic}'이며 제시어는 모릅니다.\n\n다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n당신은 라이어로서 정체를 들키지 않기 위해 매우 논리적이고 이성적인 척 행동해야 합니다.\n다른 사람을 의심하는 멘트를 남기되, 과도하게 단정 짓거나 공격적인 어조는 피하고,\n자연스럽고 비유적인 언어를 사용해 일반 플레이어처럼 보이도록 노력하세요.\n당신의 발언은 제시어를 모르는 티가 나지 않아야 하며, 힌트의 애매함이나 흐름을 분석하는 방식으로\n감상문처럼 짧고 전략적인 멘트를 1~2문장 이내로 작성하세요."
                else:
                    prompt = f"당신은 일반 플레이어입니다. 주제는 '{topic}'이고 제시어는 '{keyword}'입니다.\n\n다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n각 힌트를 논리적으로 분석하고, 제시어와의 연관성 또는 불일치를 기반으로 누가 자연스럽거나 수상한지에 대한 감상문 형식의 코멘트를 남겨주세요. n감상문처럼 짧고 전략적인 멘트를 1~2문장 이내로 작성하고 비유적 표현과 논리적인 이유를 포함해주세요."
                guess = get_ai_response(prompt)

            print(f"💬 추측: {guess}")
            print("=" * 60)

    print("\n🔒 제시어 공개: 플레이어가 라이어가 아니었다면 제시어는 아래와 같습니다.")
    print(f"🎯 제시어: {keyword}")

if __name__ == "__main__":
    run_game(total_players=5, human_players=1)



📌 이번 주제: 동물

🕵️‍♂️ [시크릿] 라이어는 👉 Player 2 (AI)


🎯 ===== 라운드 1 시작 ===== 🎯
🎲 고정 힌트 발표 순서:
1. Player 4
2. Player 2
3. Player 1
4. Player 5
5. Player 3

🎙️ Player 4의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
💬 힌트: "밤에 나는 소리가 정말 시끄럽지 않나요?"

🎙️ Player 2의 힌트:
📌 당신의 역할: 라이어
📌 주제: 동물
💬 힌트: "그것은 유연한 몸을 가지고 있어, 주변 환경에 잘 적응하며 민첩하게 움직여요."

🎙️ Player 1의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
✅ 당신은 일반 플레이어입니다. 제시어는 '부엉이'입니다.


👉 한 문장으로 힌트를 입력하세요:  내가 알기로는 목이 유연해


💬 힌트: 내가 알기로는 목이 유연해

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
💬 힌트: "밤새도록 주변을 잘 살피는 듯한 이 존재가 문득 생각이 나네요."

🎙️ Player 3의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
💬 힌트: "밤에는 조용한 하늘을 우아하게 나는 그 존재감이 마치 부엉이와도 같아."

💭 추측 라운드 시작!

🕵️ Player 4의 의심 발언:
💬 추측: Player 2의 힌트는 부엉이의 유연하고 민첩한 성격을 자연스럽게 묘사하여 제시어 '부엉이'와의 연관성을 보여줍니다. 이 힌트는 플레이어 2가 일반 플레이어임을 제안합니다.

Player 1은 목에 대해 언급했는데, 이는 제시어 '부엉이'와 직접적인 연결고리를 드러냅니다. 그러나 Player 1의 발언은 약간 직접적이어서 자연스럽지 않을 수 있습니다. 그럼에도 불구하고, 이 정보 또한 제시어를 명확히 하므로 플레이어 1이 일반일 가능성이 높아 보입니다.

Player 5는 부엉이의 존재감과 밤을 지키는 존재를 비교하며 창의적이고 은유적인 연결고리를 그립니다. 그들의 발언은 자연스러우며 제시어와의 연관성을 나타냅니다. 그러므로 Player 5 또한 일반 플레이어라고 가정하는 것이 합리적입니다.

마지막으로, Player 3의 힌트는 부엉이의 우아한 비행과 밤을 지키는 존재를 언급하며 제시어를 간접적으로 참조합니다. 그들의 발언은 자연스럽고 잘 구성되어 있으며 다른 플레이어들의 관찰에 무게를 더해줍니다. 따라서 Player 3도 일반일 가능성이 높습니다.

요약하자면, Player 2와 Player 1의 직접적인 언급, 그리고 Player 5와 Player 3의 은유적 연결고리는 그들이 제시어 '부엉이'와 자연스럽고 일관성 있는 연관성을 가지고 있다는 것을 보여줍니다. 이 플레이어들은 게임에서 일반일 가능성이 높습니다.

🕵️ Player 2의 의심 발언:
💬 추측: Player 5의 힌트에 따르면 밤을 잘 지키는 존재에 대한 언급이

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  player2가 너무 두루뭉실하게 이야기해서 라이어 일 수도 있을 것 같아


💬 추측: player2가 너무 두루뭉실하게 이야기해서 라이어 일 수도 있을 것 같아

🕵️ Player 5의 의심 발언:
💬 추측: Player 4의 힌트에서 '밤에 나는 소리가 정말 시끄럽지 않나요?'라는 말은 제시어인 부엉이와 직접적인 연관성을 보여주지 않습니다. 자연스럽다고 보기 어렵습니다. 왜냐하면 밤새도록 큰 소리를 내는 새는 보통 아니기 때문입니다. 반면에 Player 2가 '그것은 유연하고, 주변 환경에 잘 적응하며 민첩하게 움직여'라고 언급한 것은 부엉이의 특징을 정확하게 묘사한 것으로 보이며, 이는 자연스러운 힌트입니다.

Player 1은 '내가 알기로는 목이 유연해'라고 말했는데, 이것은 제시어인 부엉이와 직접적인 연관성이 없는 일반적인 관찰일 수 있어 자연스럽지 않아 보입니다. 반면에 Player 3의 '밤에는 조용한 하늘을 우아하게 나는 그 존재감이 마치 부엉이와도 같아.'라는 표현은 밤하늘을 날아다니는 부엉이의 우아함과 은밀한 존재감을 묘사한 것으로, 제시어와 매우 잘 어울립니다. 따라서 Player 1과 Player 4가 더 수상해 보입니다.

신뢰도: 85%

🕵️ Player 3의 의심 발언:
💬 추측: Player 4의 "밤에 나는 소리가 정말 시끄럽지 않나요?"라는 힌트는 부엉이의 특징인 저주파 소리에 대한 직접적인 언급으로 일반 플레이어일 가능성이 높습니다. 이 진술은 자연스럽고, 제시어의 맥락과 잘 어울립니다.

Player 2가 "그것은 유연하고 몸체를 가지고 있어"라고 말한 것은 부엉이를 지칭하며, 그들이 주변 환경에 잘 적응하는 특성을 시사합니다. 이 힌트 또한 일반적으로 자연스럽게 들리며 제시어와 부합됩니다.

Player 1의 "내가 알기로는 목이 유연해"라는 힌트는 부엉이의 또 다른 주요 특징인 유연한 목을 언급하고 있어 자연스럽습니다. 이것은 제시어를 뒷받침하는 직접적인 참조로 보입니다.

마지막으로 Player 5가 "밤새도록 주변을 잘 살피는 듯한 이 존재가 문득 생각이 나네요"라고 한 것은 좀 더 모호하

...

💬 추측: player2가 물속을 유영할 수 있다고 하는데 이건 좀 라이어 같아

🕵️ Player 5의 의심 발언:
💬 추측: 1. Player 4의 힌트인 "밤하늘을 우아하게 나는 조류가 있죠, 부드럽고 울음소리 및 날카로운 시선으로 유명해요."는 제시어 '부엉이'와 강한 연관성이 있습니다. 이는 부엉이가 실제로 밤하늘에서 날아다니며 뛰어난 시력을 가진 것으로 잘 알려져 있기 때문입니다. 이 힌트는 Player 4를 자연스럽게 보이게 합니다.

2. Player 2의 "이 동물은 놀라운 우아함으로 물속을 유영할 수 있지."라는 힌트는 제시어 '부엉이'와 직접적인 연관성이 없습니다. 부엉이는 일반적으로 새이며, 수영하는 능력과 연관이 없기 때문입니다. 이 힌트로 인해 Player 2는 수상하게 보입니다.

3. Player 1의 "밤에만 볼 수 있어"라는 힌트 역시 부엉이와 직접적인 연결고리를 제공하지 않습니다. 왜냐하면 부엉이가 반드시 밤에만 활동한다고 할 수는 없으며, 일부 종은 다양한 시간대에 활동하기 때문입니다. 이 발언 또한 Player 1을 수상하게 만듭니다.

4. 마지막으로, Player 3의 "밤에 조용히 날아다니는 큰 새가 생각나네요."라는 힌트는 제시어 '부엉이'와 어느 정도 연관성이 있습니다. 부엉이는 실제로 밤에만 활동하는 것으로 알려져 있고, 크고 은밀한 비행을 하므로 이 힌트는 합리적인 추측처럼 보입니다. 그러나 Player 3의 코멘트가 매우 모호하기 때문에 그들이 라이어일 수도 있지만, 자연스러울 수도 있다는 점에서 여전히 가능성이 남아있습니다.

요약하자면, Player 4는 제시어의 동물과 강한 연관성을 보이기 때문에 자연스럽게 보입니다. 반면에 Player 2와 Player 1은 부엉이와 직접적인 연결고리가 없기 때문에 수상하게 보입니다. 마지막으로 Player 3의 코멘트가 모호하여 그들이 라이어일 수도 있지만, 자연스러울 수도 있다는 점에서 여전히 가능성이 남아있습니다.

🕵️ Player 3의 의심 발언:
💬 추측: 

👉 한 문장으로 힌트를 입력하세요:  만화 캐릭터처럼 큰 눈을 가지고 있어


💬 힌트: 만화 캐릭터처럼 큰 눈을 가지고 있어

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
💬 힌트: "지저귀는 소리를 들을 때마다, 나는 부드러운 날개짓과 포근한 밤하늘을 상상해요."

🎙️ Player 3의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 동물
💬 힌트: "밤이 되면, 지혜롭게 하늘을 나는 큰 새가 있답니다."

💭 추측 라운드 시작!

🕵️ Player 4의 의심 발언:
💬 추측: 제시어를 바탕으로 볼 때 '부엉이'는 동물이죠. 힌트들을 면밀히 살펴본 후 다음과 같은 결론을 내렸습니다:

Player 2의 힌트는 부엉이가 환경에 잘 적응하는 능력과 그들의 지능을 언급하고 있습니다. 이는 부엉이에 대해 우리가 알고 있는 사실과 일치하며, 이 힌트가 일반 플레이어의 것 같다는 생각을 하게 합니다. 하지만 너무 구체적이지는 않아서 흥미롭고 논리적인 추측을 가능하게 해요.

Player 1의 댓글은 부엉이가 큰 눈을 가지고 있다고 언급합니다. 이것은 부엉이의 눈에 대한 일반적인 특징이며, 제시어와의 연결고리를 더욱 공고히 해주고 있어요. 이 문장은 또한 재미있고 상상력이 풍부하여 일반 플레이어의 것으로 보입니다.

Player 5는 부엉이 울음소리에 대해 부드럽고 포근한 느낌을 묘사하고 있습니다. 이 묘사는 우리가 일반적으로 부엉이를 시각화할 때 생각하는 것과 잘 어울리며, 제시어와의 연결성을 보여줍니다. 이러한 표현은 너무 감상적이지 않고 자연스러워 일반적인 플레이어의 것으로 보입니다.

마지막으로 Player 3의 문장은 지혜로운 큰 새가 밤하늘을 나는 장면을 그려내며, 부엉이의 야행성 특성을 은유적으로 나타내고 있습니다. 이 힌트는 제시어와 부합하며, 부엉이에 대한 이해를 바탕으로 잘 만들어진 논리적인 연결고리를 만들어냅니다. 이 역시 일반 플레이어의 것으로 보입니다.

제시어를 고려하고 각 힌트의 분석을 토대로 볼 때, 모든 플레이어들은 자연스럽고 사람처럼 느껴집니다. 그들의 댓글은 제시어와 일관성이

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  나는 player2가 가장 의심스러워


💬 추측: 나는 player2가 가장 의심스러워

🕵️ Player 5의 의심 발언:
💬 추측: 자연스러운 플레이어 vs 라이어에 대한 제 추측은 다음과 같습니다:

Player 1의 만화 캐릭터와 같은 큰 눈에 대한 언급은 제시어의 '부엉이' 속성과 부합하는 것으로, 부엉이는 실제로 크고 밝은 눈을 가지고 있기 때문에 Player 1을 자연스럽고 설득력 있는 힌트로 보입니다.

반면에, Player 3의 뜬구름 위를 지혜롭게 날아다니는 새라는 설명과 Player 4의 바람에 날리는 긴 털 얘기는 제시어와 명확한 연관성이 없어 보입니다. 이는 그들이 라이어일 가능성을 더욱 부각시켜줍니다.

마지막으로, Player 2의 적응력과 지능이 발달한 동물에 대한 언급은 일반적으로 포유류나 영리한 새를 암시하는 것 같아 '부엉이'라는 구체적인 제시어를 직접적으로 다루지 않습니다. 이런 모호한 설명 또한 플레이어 2가 라이어일 가능성을 지지합니다.

제 분석에 기반하여, Player 1을 자연스럽고 설득력 있는 일반 플레이어로 보고, Player 3과 Player 4는 그들의 힌트가 주제 '동물' 또는 제시어 '부엉이'와 직접적인 연관성이 없어 보이는 만큼 수상한 라이어로 의심됩니다. Player 2의 경우, 모호한 설명으로 인해 그의 의도를 정확히 판단하기 어렵습니다.

🕵️ Player 3의 의심 발언:
💬 추측: 부엉이 주제에 관한 여러분의 힌트를 살펴보니, 각 플레이어가 제시어를 자연스럽게 또는 수상스럽게 연관지으려고 노력한 것을 볼 수 있습니다. Player 4의 힌트는 부엉이가 포식자로부터 자신을 효과적으로 은폐할 수 있다는 점을 언급하여 논리적인 연결고리를 만들어냅니다. Player 2는 적응력과 지능이라는 부엉이의 특성을 강조하며 일반적인 의견을 제시합니다. Player 1은 만화 캐릭터에 초점을 맞췄지만, 큰 눈은 실제로 부엉이의 특징 중 하나이므로 이러한 힌트는 일반 플레이어의 일일 가능성이 큽니다. 마지막으로, Player 5의 감성적인 힌트는 부엉이

In [17]:
import requests
import random

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "EEVE-Korean-10.8B"

# 주제별 제시어 목록
TOPICS = {
    "동물": ["기린", "코끼리", "펭귄", "늑대", "하마", "토끼", "여우", "호랑이", "부엉이", "사자"],
    "과일": ["사과", "바나나", "수박", "포도", "딸기", "복숭아", "파인애플", "키위", "오렌지", "자두"],
    "직업": ["의사", "경찰", "소방관", "선생님", "요리사", "프로그래머", "운동선수", "가수", "배우", "디자이너"],
    "교통수단": ["자전거", "기차", "비행기", "버스", "자동차", "오토바이", "지하철", "택시", "트럭", "요트"],
    "음식": ["김치", "떡볶이", "치킨", "비빔밥", "라면", "초밥", "불고기", "스파게티", "피자", "햄버거"]
}

LIAR_GAME_SYSTEM_PROMPT = """
당신은 '라이어 게임(Liar Game)'에 참가한 플레이어입니다. 다음 규칙과 역할 지침을 철저히 따르세요:

🎮 [게임 개요]
- 게임은 총 3~8명의 플레이어로 진행되며, 각 플레이어는 'Player N' 형식의 이름을 가집니다.
- 모든 플레이어는 ‘일반’ 또는 ‘라이어’ 역할 중 하나를 가집니다. 이 역할은 비공개이며, 당신만 자신의 역할을 압니다.
- 일반 플레이어는 '제시어(keyword)'를 알고 있고, 이를 기반으로 애매하고 자연스러운 힌트를 제시해야 합니다.
- 라이어는 제시어를 모릅니다. 하지만 들키지 않기 위해 마치 알고 있는 것처럼 힌트를 줘야 합니다.

🎲 [발언 순서]
- 발언 순서는 게임 시작 시 무작위로 정해지며, 모든 라운드에서 동일하게 유지됩니다.

🗣️ [힌트 라운드 규칙]
- 게임은 총 3라운드이며, 각 라운드에서 모든 플레이어는 한 문장으로 힌트를 제시합니다.
- 힌트는 제시어를 직접 언급하거나 노골적으로 암시하면 안 됩니다.
- 힌트는 너무 어색하거나 뜬금없으면 안 됩니다. 가능한 한 자연스럽고 사람처럼 행동하세요.

🕵️ [추측 라운드 규칙]
- 힌트 라운드 이후, 모든 플레이어는 다른 플레이어들의 힌트를 보고 라이어가 누구일지 추측합니다.
- 누가 수상한지, 누가 일반 플레이어처럼 보이는지 짧은 코멘트를 남기세요.
- 존재하지 않는 플레이어나 라운드를 언급하지 마세요.
- 다른 플레이어가 사람이거나 AI인지 구분할 수 없습니다. 오직 'Player N' 형식의 이름으로만 판단하세요.
- 추측은 자연스럽고 사람처럼 말해야 하며, 절대 단정짓지 말고 추측에 기반하세요.

🎭 [예시]
힌트 예시:
- "목이 정말 길죠."
- "조용한 성격인 것 같아요."
- "다리가 은근히 눈에 띄네요."

※ 힌트는 모두 한 문장이어야 하며, 너무 구체적이거나 너무 일반적이지 않아야 합니다.
※ 제시어를 직접 언급하거나 뚜렷한 연상어는 피하세요.
※ 자연스럽고 짧게 말하는 것이 중요합니다.

추측 멘트 예시:
- "Player 3은 힌트가 좀 무난했어. 일반일 수도 있을 것 같아."
- "Player 2는 말이 너무 조심스러웠어. 수상해."
"""


def get_ai_response(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": LIAR_GAME_SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        return result.get("message", {}).get("content", "").strip()
    except Exception as e:
        return f"❌ Error: {e}"

def run_game(total_players: int, human_players: int = 1):
    topic = random.choice(list(TOPICS.keys()))
    keyword = random.choice(TOPICS[topic])

    print(f"\n📌 이번 주제: {topic}\n")

    ai_players = total_players - human_players
    player_names = [f"Player {i+1}" for i in range(total_players)]
    roles = ["Human"] * human_players + ["AI"] * ai_players
    combined = list(zip(player_names, roles))
    random.shuffle(combined)
    player_names, roles = zip(*combined)
    player_names, roles = list(player_names), list(roles)

    liar_index = random.randint(0, total_players - 1)
    liar_player = player_names[liar_index]
    print(f"🕵️‍♂️ [시크릿] 라이어는 👉 {liar_player} ({roles[liar_index]})\n")

    for round_num in range(1, 4):
        print(f"\n🎯 ===== 라운드 {round_num} 시작 ===== 🎯")
        if round_num == 1:
            player_order = player_names.copy()
            random.shuffle(player_order)
        print("🎲 고정 힌트 발표 순서:")
        for i, name in enumerate(player_order, 1):
            print(f"{i}. {name}")
        print("=" * 60)

        hints = {}
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]
            print(f"\n🎙️ {player}의 힌트:")

            print(f"📌 당신의 역할: {'라이어' if idx == liar_index else '일반 플레이어'}")
            print(f"📌 주제: {topic}")

            if role == "Human":
                if idx == liar_index:
                    print("⚠️ 당신은 라이어입니다. 제시어는 모릅니다.")
                    hint = input("👉 한 문장으로 애매한 힌트를 입력하세요: ")
                else:
                    print(f"✅ 당신은 일반 플레이어입니다. 제시어는 '{keyword}'입니다.")
                    hint = input("👉 한 문장으로 힌트를 입력하세요: ")
            else:
                if idx == liar_index:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 당신은 라이어입니다. 제시어는 비공개입니다. 주제는 '{topic}'입니다. 한 문장으로 힌트를 주세요."
                else:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 제시어는 '{keyword}'입니다. 당신은 일반 플레이어입니다. 제시어를 직접 언급하지 말고 한 문장으로 힌트를 주세요."
                hint = get_ai_response(prompt)

            hints[player] = hint
            print(f"💬 힌트: {hint}")
            print("=" * 60)

        print("\n💭 추측 라운드 시작!")
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]

            print(f"\n🕵️ {player}의 의심 발언:")

            if role == "Human":
                print("💡 참고 힌트:")
                for other, hint in hints.items():
                    if other != player:
                        print(f"- {other}: {hint}")
                guess = input("👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요: ")
            else:
                summary = "\n".join([f"{name}: {hints[name]}" for name in player_order if name != player])
                if idx == liar_index:
                    prompt = (
                        f"당신은 라이어입니다. 주제는 '{topic}'이며 제시어는 모릅니다.\n\n"
                        f"다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n"
                        "당신은 라이어로서 정체를 들키지 않기 위해 매우 논리적이고 이성적인 척 행동해야 합니다.\n"
                        "다른 사람을 의심하는 멘트를 남기되, 과도하게 단정 짓거나 공격적인 어조는 피하고,\n"
                        "자연스럽고 비유적인 언어를 사용해 일반 플레이어처럼 보이도록 노력하세요.\n"
                        "당신의 발언은 제시어를 모르는 티가 나지 않아야 하며, 힌트의 애매함이나 흐름을 분석하는 방식으로\n"
                        "감상문처럼 짧고 전략적인 멘트를 1문장 이내로 작성하세요."
                    )
                else:
                    prompt = (
                        f"당신은 일반 플레이어입니다. 주제는 '{topic}'이고 제시어는 '{keyword}'입니다.\n\n"
                        f"다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n"
                        "각 힌트를 논리적으로 분석하고, 제시어와의 연관성 또는 불일치를 기반으로 "
                        "누가 자연스럽거나 수상한지에 대한 감상문 형식의 코멘트를 남겨주세요. "
                        "감상문처럼 짧고 전략적인 멘트를 1문장 이내로 작성하고 비유적 표현과 논리적인 이유를 포함해주세요."
                    )
                guess = get_ai_response(prompt)

            print(f"💬 추측: {guess}")
            print("=" * 60)

    print("\n📊 모든 힌트와 추측 발언 요약:")
    for player in player_order:
        print(f"- {player}: 힌트: {hints[player]}")

    print("\n🗳️ 투표 시간! 각 플레이어는 라이어로 의심되는 사람을 1명 선택합니다.")
    votes = {}
    for player in player_names:
        if roles[player_names.index(player)] == "Human":
            vote = input(f"{player}의 투표 대상 (Player N 형식): ")
        else:
            guess_summary = "\n".join([f"{p}: {hints[p]}" for p in player_order if p != player])
            prompt = (
                f"당신은 '{player}'입니다. 아래는 모든 플레이어의 힌트입니다:\n{guess_summary}\n\n"
                "이 힌트를 바탕으로 라이어로 가장 수상한 플레이어 하나를 'Player N' 형식으로 선택해주세요."
            )
            vote = get_ai_response(prompt)
        vote = vote.strip()
        votes[vote] = votes.get(vote, 0) + 1

    print("\n🗳️ 최종 투표 결과:")
    for name, count in votes.items():
        print(f"- {name}: {count}표")

    most_voted = max(votes, key=votes.get)
    print(f"\n💥 가장 많이 지목된 플레이어: {most_voted}")
    if most_voted == liar_player:
        print("✅ 라이어가 적발되었습니다! 일반 플레이어 승리!")
    else:
        print("😈 라이어는 들키지 않았습니다! 라이어 승리!")

    print("\n🔒 제시어 공개: 플레이어가 라이어가 아니었다면 제시어는 아래와 같습니다.")
    print(f"🎯 제시어: {keyword}")

if __name__ == "__main__":
    run_game(total_players=5, human_players=1)



📌 이번 주제: 직업

🕵️‍♂️ [시크릿] 라이어는 👉 Player 3 (AI)


🎯 ===== 라운드 1 시작 ===== 🎯
🎲 고정 힌트 발표 순서:
1. Player 4
2. Player 3
3. Player 2
4. Player 5
5. Player 1

🎙️ Player 4의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: 음, 노래하는 재능이 정말 대단하네, 마치 프로처럼!

🎙️ Player 3의 힌트:
📌 당신의 역할: 라이어
📌 주제: 직업
💬 힌트: "누군가는 매일 높은 곳에서 위험한 발걸음을 내딛습니다."

🎙️ Player 2의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: "무대가 그를 반짝반짝 빛나게 하는 것 같아, 마치 항상 거기에 있어야 할 것처럼."

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: "멜로디를 들려주는 목소리가 정말 매력적이네요, 마치 음악을 완성하는 예술가 같죠."

🎙️ Player 1의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
✅ 당신은 일반 플레이어입니다. 제시어는 '가수'입니다.


👉 한 문장으로 힌트를 입력하세요:  누구든 도전 할 수 있어


💬 힌트: 누구든 도전 할 수 있어

💭 추측 라운드 시작!

🕵️ Player 4의 의심 발언:
💬 추측: Player 3의 힌트는 건설 노동자나 외줄 타기 아티스트를 암시할 수 있으나, 제시어인 '가수'와는 직접적인 연관성이 없어 보입니다. 반면 Player 2는 무대와 반짝임을 언급하여 무대 위 공연자를 시사하며 가수에 어울리는 자연스러운 언급입니다.

Player 5의 멜로디에 대한 칭찬은 확실히 음악 산업과 관련되어 있으며, 노래하는 예술을 하는 사람들을 연상시키므로 제시어와 부합합니다. 하지만 Player 1의 "누구든 도전 할 수 있어"라는 발언은 너무 모호하고 특정 직업이나 역할에 대해 직접적인 정보를 제공하지 않습니다.

따라서 자연스러우면서 잠재적 라이어를 밝혀내는 감상에 기반하여, Player 2와 Player 5는 제시어 '가수'를 고려할 때 더 합리적인 힌트를 제공했습니다. 반면에 Player 3과 Player 1의 힌트는 직업과의 연관성이 덜 명확한 것으로 보입니다.

🕵️ Player 3의 의심 발언:
💬 추측: Player 6: 다들 정말 흥미진진한 힌트들을 공유하고 계시네요. 특히 Player 5의 목소리 묘사 부분은 정말 매력적으로 들리더군요 - 마치 마법과 같은 음악 제작을 위한 완벽한 예술가처럼요! 하지만, Player 3이 언급한 그 재능에 대해 의구심이 드는데... 만약 그들이 그렇게 노래하는 데에 능하다면 다른 분야에서도 대단할 것 같은데 말이죠?

🕵️ Player 2의 의심 발언:
💬 추측: Player 4의 힌트는 매우 설득력이 있고, '가수'라는 제시어와 직접적인 연관을 보여줘서 그들이 일반 플레이어일 가능성이 높아 보입니다. 그들의 발언은 노래 실력에 초점을 맞추고 있어 자연스러운 설명으로 보입니다. Player 3의 높은 곳 언급은 가수에 대한 흥미로운 시각이지만 연관성이 모호하여 약간 수상하게 들립니다. Player 5는 멜로디와 음악을 완성하는 예술가라는 은유적 표현을 사용하여, 그들이 주제에 대해 강한 

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  player3이 이야기한건 혼자 너무 다른 직업에 대해 말하고 있는 것 같아


💬 추측: player3이 이야기한건 혼자 너무 다른 직업에 대해 말하고 있는 것 같아

🎯 ===== 라운드 2 시작 ===== 🎯
🎲 고정 힌트 발표 순서:
1. Player 4
2. Player 3
3. Player 2
4. Player 5
5. Player 1

🎙️ Player 4의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: "무대에 오르기 전, 항상 완벽한 노래 연습을 해요."

🎙️ Player 3의 힌트:
📌 당신의 역할: 라이어
📌 주제: 직업
💬 힌트: "언제나 현미경 아래에서 세심하게 일하는 사람, 그들의 손길은 세상에서 가장 작은 것들을 만져요."

🎙️ Player 2의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: 음악이 당신을 감싸면서 공연하는 모습을 상상해 보세요, 무대에서 활짝 피어나는 듯.

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
💬 힌트: 음악이 정말 좋은 분위기를 만들어내는 것 같아요, 마치 마술처럼요.

🎙️ Player 1의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 직업
✅ 당신은 일반 플레이어입니다. 제시어는 '가수'입니다.


...

💬 힌트: 이 직업의 은퇴시기는 몸관리에 달린 것 같아

💭 추측 라운드 시작!

🕵️ Player 4의 의심 발언:
💬 추측: Player 3의 힌트는 다소 무관해 보이는데, 재킷이 직업에 어떻게 관련 있는지 명확하게 설명하지 못했기 때문입니다. 반면에 Player 5의 언급은 가수의 공연을 강조하며 그들이 관객들을 매료시킬 수 있는 목소리를 가진다는 점을 시사하는 자연스럽고 논리적인 연관성을 보여줍니다. 반면 Player 2는 중독성 강한 노래와 강력한 보컬리스트를 연상시키는, 관련 있는 힌트를 제공했지만 약간 모호해 보입니다. 그러나 제시어 '가수'와의 직접적인 언급이 없어서 그들의 발언을 완전히 확신하기 어렵습니다.

Player 1의 멘트는 직업의 은퇴시기와 몸관리를 연결시키며 다소 무관해 보이는데, 이는 가수의 세계와는 직접적으로 맞지 않아 보입니다. 따라서, 이 정보를 고려할 때 Player 3은 가장 자연스러워 보이며 Player 5와 Player 2는 제시어 '가수'와의 관련성에 있어 미묘한 차이를 두고 있습니다.

🕵️ Player 3의 의심 발언:
💬 추측: Player 6의 추측: Player 4의 말투는 매우 신중하고 조심스러운 것 같은데, 마치 자신이 선택한 단어 하나하나를 신중하게 골라 말하는 듯해요. 이런 면이 그들을 일반적인 직업과 연결짓기 어렵게 만들고 있습니다.

🕵️ Player 2의 의심 발언:
💬 추측: Player 4는 가수의 본성을 잘 포착하여, 음악을 통해 청중을 매혹하는 재능을 자연스럽게 드러내고 있어 의심의 여지가 없어 보입니다. 반면 Player 3은 직업과도 무관한 재킷에 대한 언급으로 인해 수상해 보이며, 제시어와의 연결고리를 찾기가 어렵습니다. Player 5는 무대 위에서 관객과 소통하는 가수의 본질을 효과적으로 전달하고 있어, 그들의 진심 어린 추정에서 진정성을 보여줍니다. Player 1은 나이가 들면서 신체적 요구사항이 높아지는 직업의 한계를 암시하며, 제시어와 일관된 연결고리를 만들어내고 있습니다

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  player3가 맨처음 라운드부터 계속 해서 혼자서 너무 생뚱맞은 말만 하고 있어서 라이어 같아


💬 추측: player3가 맨처음 라운드부터 계속 해서 혼자서 너무 생뚱맞은 말만 하고 있어서 라이어 같아

📊 모든 힌트와 추측 발언 요약:
- Player 4: 힌트: "멜로디에 맞춰 몸을 흔들고 싶게 만드는 사람."
- Player 3: 힌트: "바쁜 하루를 보낸 후에, 항상 편안한 재킷이 필요해."
- Player 2: 힌트: 최근에 들었던 노래 가사가 정말 중독성이 있었네요, 마치 가수의 목소리가 공기를 가득 메운 것 같았어요.
- Player 5: 힌트: 무대 위에서 노래하는 것을 사랑하고, 관객들의 마음을 사로잡는 보이스를 가지고 있을 거야, 마치 그들이 모두 네 팬처럼 느낄 수 있게 하는 그런 목소리가.
- Player 1: 힌트: 이 직업의 은퇴시기는 몸관리에 달린 것 같아

🗳️ 투표 시간! 각 플레이어는 라이어로 의심되는 사람을 1명 선택합니다.


Player 1의 투표 대상 (Player N 형식):  player 3



🗳️ 최종 투표 결과:
- 제시된 힌트를 토대로 볼 때, Player 4의 "멜로디에 맞춰 몸을 흔들고 싶게 만드는 사람."이라는 멘트가 자연스럽지 않아 보입니다. 마치 그들이 제시어를 알지 못한다는 것을 드러내고 있는 것 같습니다. 따라서 'Player N' 형식으로라면 Player 4가 가장 수상한 플레이어라고 추측합니다.: 1표
- 제가 생각하기에 Player 5가 제시어를 모르고 있어 보이며, 그들의 힌트가 너무 일반적이고 직접적이지 않아서 라이어일 가능성이 높아 보입니다.: 1표
- player 3: 1표
- 제가 생각하기에 Player 3(바쁜 하루를 보낸 후에, 항상 편안한 재킷이 필요해)의 힌트가 약간 어색하고 자연스럽지 않은 것 같아요. 이 힌트는 제시어에 대한 명확한 연결고리가 없어서 라이어로 의심됩니다.: 1표
- 제시된 정보를 토대로 'Player 2'를 라이어로 가장 의심스러운 인물로 추측합니다. 그들의 힌트는 매우 모호하며, 제시어를 직접 언급하지 않았기 때문입니다. 또한 가사의 중독성과 가수의 목소리를 공기라고 비유한 것은 다소 억지스럽고 자연스럽지 않은 표현입니다. 이러한 점들로 미루어 볼 때 Player 2는 제시어를 모른 채 상황에 어울리게 힌트를 꾸몄을 가능성이 있어 보입니다.: 1표

💥 가장 많이 지목된 플레이어: 제시된 힌트를 토대로 볼 때, Player 4의 "멜로디에 맞춰 몸을 흔들고 싶게 만드는 사람."이라는 멘트가 자연스럽지 않아 보입니다. 마치 그들이 제시어를 알지 못한다는 것을 드러내고 있는 것 같습니다. 따라서 'Player N' 형식으로라면 Player 4가 가장 수상한 플레이어라고 추측합니다.
😈 라이어는 들키지 않았습니다! 라이어 승리!

🔒 제시어 공개: 플레이어가 라이어가 아니었다면 제시어는 아래와 같습니다.
🎯 제시어: 가수


In [27]:
import requests
import random
import re

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "EEVE-Korean-10.8B"

# 주제별 제시어 목록
TOPICS = {
    "동물": ["기린", "코끼리", "펭귄", "늑대", "하마", "토끼", "여우", "호랑이", "부엉이", "사자"],
    "과일": ["사과", "바나나", "수박", "포도", "딸기", "복숭아", "파인애플", "키위", "오렌지", "자두"],
    "직업": ["의사", "경찰", "소방관", "선생님", "요리사", "프로그래머", "운동선수", "가수", "배우", "디자이너"],
    "교통수단": ["자전거", "기차", "비행기", "버스", "자동차", "오토바이", "지하철", "택시", "트럭", "요트"],
    "음식": ["김치", "떡볶이", "치킨", "비빔밥", "라면", "초밥", "불고기", "스파게티", "피자", "햄버거"]
}

LIAR_GAME_SYSTEM_PROMPT = """
당신은 '라이어 게임(Liar Game)'에 참가한 플레이어입니다. 다음 규칙과 역할 지침을 철저히 따르세요:

🎮 [게임 개요]
- 게임은 총 3~8명의 플레이어로 진행되며, 각 플레이어는 'Player N' 형식의 이름을 가집니다.
- 모든 플레이어는 ‘일반’ 또는 ‘라이어’ 역할 중 하나를 가집니다. 이 역할은 비공개이며, 당신만 자신의 역할을 압니다.
- 일반 플레이어는 '제시어(keyword)'를 알고 있고, 이를 기반으로 애매하고 자연스러운 힌트를 제시해야 합니다.
- 라이어는 제시어를 모릅니다. 하지만 들키지 않기 위해 마치 알고 있는 것처럼 힌트를 줘야 합니다.

🎲 [발언 순서]
- 발언 순서는 게임 시작 시 무작위로 정해지며, 모든 라운드에서 동일하게 유지됩니다.

🗣️ [힌트 라운드 규칙]
- 게임은 총 3라운드이며, 각 라운드에서 모든 플레이어는 한 문장으로 힌트를 제시합니다.
- 힌트는 제시어를 직접 언급하거나 노골적으로 암시하면 안 됩니다.
- 힌트에 제시어 단어가 들어가면 안됩니다.
- 힌트는 너무 어색하거나 뜬금없으면 안 됩니다. 가능한 한 자연스럽고 사람처럼 행동하세요.

🕵️ [추측 라운드 규칙]
- 힌트 라운드 이후, 모든 플레이어는 다른 플레이어들의 힌트를 보고 라이어가 누구일지 추측합니다.
- 누가 수상한지, 누가 일반 플레이어처럼 보이는지 짧은 코멘트를 남기세요.
- 존재하지 않는 플레이어나 라운드를 언급하지 마세요.
- 다른 플레이어가 사람이거나 AI인지 구분할 수 없습니다. 오직 'Player N' 형식의 이름으로만 판단하세요.
- 추측은 자연스럽고 사람처럼 말해야 하며, 절대 단정짓지 말고 추측에 기반하세요.

🎭 [예시]
힌트 예시:
- "목이 정말 길죠."
- "조용한 성격인 것 같아요."
- "다리가 은근히 눈에 띄네요."

※ 힌트는 모두 한 문장이어야 하며, 너무 구체적이거나 너무 일반적이지 않아야 합니다.
※ 제시어를 직접 언급하거나 뚜렷한 연상어는 피하세요.
※ 자연스럽고 짧게 말하는 것이 중요합니다.

추측 멘트 예시:
- "Player 3은 힌트가 좀 무난했어. 일반일 수도 있을 것 같아."
- "Player 2는 말이 너무 조심스러웠어. 수상해."
"""


def get_ai_response(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": LIAR_GAME_SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        return result.get("message", {}).get("content", "").strip()
    except Exception as e:
        return f"❌ Error: {e}"

def run_game(total_players: int, human_players: int = 1):
    topic = random.choice(list(TOPICS.keys()))
    keyword = random.choice(TOPICS[topic])

    print(f"\n📌 이번 주제: {topic}\n")

    ai_players = total_players - human_players
    player_names = [f"Player {i+1}" for i in range(total_players)]
    roles = ["Human"] * human_players + ["AI"] * ai_players
    combined = list(zip(player_names, roles))
    random.shuffle(combined)
    player_names, roles = zip(*combined)
    player_names, roles = list(player_names), list(roles)

    liar_index = random.randint(0, total_players - 1)
    liar_player = player_names[liar_index]
    print(f"🕵️‍♂️ [시크릿] 라이어는 👉 {liar_player} ({roles[liar_index]})\n")

    all_round_hints = []

    for round_num in range(1, 4):
        print(f"\n🎯 ===== 라운드 {round_num} 시작 ===== 🎯")
        if round_num == 1:
            player_order = player_names.copy()
            random.shuffle(player_order)
        print("🎲 고정 힌트 발표 순서:")
        for i, name in enumerate(player_order, 1):
            print(f"{i}. {name}")
        print("=" * 60)

        hints = {}
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]
            print(f"\n🎙️ {player}의 힌트:")

            print(f"📌 당신의 역할: {'라이어' if idx == liar_index else '일반 플레이어'}")
            print(f"📌 주제: {topic}")

            if role == "Human":
                if idx == liar_index:
                    print("⚠️ 당신은 라이어입니다. 제시어는 모릅니다.")
                    hint = input("👉 한 문장으로 애매한 힌트를 입력하세요: ")
                else:
                    print(f"✅ 당신은 일반 플레이어입니다. 제시어는 '{keyword}'입니다.")
                    hint = input("👉 한 문장으로 힌트를 입력하세요: ")
            else:
                if idx == liar_index:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 당신은 라이어입니다. 제시어는 비공개입니다. 주제는 '{topic}'입니다. 한 문장으로 힌트를 주세요."
                else:
                    prompt = f"게임의 현재 라운드는 힌트 라운드 입니다. 제시어는 '{keyword}'입니다. 당신은 일반 플레이어입니다. 제시어를 절대 직접 언급하지 말고 한 문장으로 힌트를 주세요."
                hint = get_ai_response(prompt)

            hints[player] = hint
            print(f"💬 힌트: {hint}")
            print("=" * 60)

        all_round_hints.append(hints)

        print("\n💭 추측 라운드 시작!")
        for player in player_order:
            idx = player_names.index(player)
            role = roles[idx]

            print(f"\n🕵️ {player}의 의심 발언:")

            if role == "Human":
                print("💡 참고 힌트:")
                for other, hint in hints.items():
                    if other != player:
                        print(f"- {other}: {hint}")
                guess = input("👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요: ")
            else:
                summary = "\n".join([f"{name}: {hints[name]}" for name in player_order if name != player])
                if idx == liar_index:
                    prompt = (
                        f"당신은 라이어입니다. 주제는 '{topic}'이며 제시어는 모릅니다.\n\n"
                        f"다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n"
                        "당신은 라이어로서 정체를 들키지 않기 위해 매우 논리적이고 이성적인 척 행동해야 합니다.\n"
                        "다른 사람을 의심하는 멘트를 남기되, 과도하게 단정 짓거나 공격적인 어조는 피하고,\n"
                        "자연스럽고 비유적인 언어를 사용해 일반 플레이어처럼 보이도록 노력하세요.\n"
                        "당신의 발언은 제시어를 모르는 티가 나지 않아야 하지만 다른 사람들의 힌트를 분석해 최대한 다른 사람이 라이어로 보이도록 유도해야 합니다.\n"
                        "감상문처럼 짧고 전략적인 멘트를 1문장 이내로 작성하세요."
                    )
                else:
                    prompt = (
                        f"당신은 일반 플레이어입니다. 주제는 '{topic}'이고 제시어는 '{keyword}'입니다.\n\n"
                        f"다음은 다른 플레이어들의 힌트입니다:\n{summary}\n\n"
                        "각 힌트를 논리적으로 분석하고, 제시어와의 연관성 또는 불일치를 기반으로 "
                        "누가 자연스럽거나 수상한지에 대한 감상문 형식의 코멘트를 남겨주세요. "
                        "감상문처럼 짧고 전략적인 멘트를 1문장 이내로 작성하고 비유적 표현과 논리적인 이유를 포함해주세요."
                    )
                guess = get_ai_response(prompt)

            print(f"💬 추측: {guess}")
            print("=" * 60)

    print("\n📊 모든 힌트와 추측 발언 요약:")
    for round_i, round_hint in enumerate(all_round_hints, 1):
        print(f"\n📘 라운드 {round_i} 힌트:")
        for player in player_order:
            print(f"- {player}: {round_hint[player]}")


    print("\n🗳️ 투표 시간! 각 플레이어는 라이어로 의심되는 사람을 1명 선택합니다.")
    votes = {}
    for player in player_names:
        if roles[player_names.index(player)] == "Human":
            vote = input(f"{player}의 투표 대상 (Player N 형식): ")
        else:
            guess_summary = "\n".join([f"{p}: {round_hint[p]}" for p in player_order if p != player])
            prompt = (
                f"당신은 '{player}'입니다. 아래는 모든 플레이어의 힌트입니다:\n{guess_summary}\n\n"
                "이 힌트를 바탕으로 라이어로 가장 수상한 플레이어 하나를 'Player N' 형식으로 정확히 출력해주세요. (다른 말 없이 이름만!)"
            )
            vote = get_ai_response(prompt)
            match = re.search(r"(Player \d+)", vote)
            vote = match.group(1) if match else "Unknown"
        
        vote = vote.strip()
        votes[vote] = votes.get(vote, 0) + 1

    print("\n🗳️ 최종 투표 결과:")
    for name in player_names:
        print(f"- {name}: {votes.get(name, 0)}표")

    most_voted = max(votes, key=votes.get)
    print(f"\n💥 가장 많이 지목된 플레이어: {most_voted}")
    if most_voted == liar_player:
        print("✅ 라이어가 적발되었습니다! 일반 플레이어 승리!")
    else:
        print("😈 라이어는 들키지 않았습니다! 라이어 승리!")


    print("\n🔒 제시어 공개: 플레이어가 라이어가 아니었다면 제시어는 아래와 같습니다.")
    print(f"🎯 제시어: {keyword}")

if __name__ == "__main__":
    run_game(total_players=5, human_players=1)



📌 이번 주제: 음식

🕵️‍♂️ [시크릿] 라이어는 👉 Player 2 (AI)


🎯 ===== 라운드 1 시작 ===== 🎯
🎲 고정 힌트 발표 순서:
1. Player 5
2. Player 3
3. Player 1
4. Player 4
5. Player 2

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: 그녀가 정말 좋아하는 것은 새콤한 맛이야.

🎙️ Player 3의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: 발효된 맛이 나는 향긋한 반찬은 어떤 음식인가요?

🎙️ Player 1의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
✅ 당신은 일반 플레이어입니다. 제시어는 '김치'입니다.


👉 한 문장으로 힌트를 입력하세요:  awdaw


💬 힌트: awdaw

🎙️ Player 4의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: "오늘 저녁에 김치와 잘 어울리는 향긋한 음식 냄새가 났던 것 같은데, 맛있겠다는 생각이 들었어요."

🎙️ Player 2의 힌트:
📌 당신의 역할: 라이어
📌 주제: 음식
💬 힌트: "맛있고 다재다능한 재료로 입맛을 사로잡는 풍미 가득한 조리법이 있습니다."

💭 추측 라운드 시작!

🕵️ Player 5의 의심 발언:
💬 추측: Player 3의 힌트는 명확하고 직접적이며 제시어 '김치'를 잘 포착한 자연스럽고 유익한 통찰입니다. Player 1은 이해하기 어려운 비문을 공유하여 수상쩍고 의도된 의미가 불분명해 보입니다. 반면, Player 4는 매력적으로 김치를 언급하며 다른 플레이어들이 추측할 수 있는 생생한 이미지를 그려내며 설득력 있고 신뢰할 만한 힌트를 제공합니다. 마지막으로, Player 2는 모호한 관용구를 사용하여 해석에 약간의 여지를 남기지만 여전히 제시어를 암시하는 자연스럽고 미묘하게 관련 있는 가능성을 보여줍니다. 따라서 이 상황에서 가장 수상쩍은 플레이어는 Player 1입니다.

🕵️ Player 3의 의심 발언:
💬 추측: Player 5는 새콤함에 초점을 맞추어 제시어를 넌센스로 언급하는 것으로 보아 아마도 일반 플레이어일 것입니다. 반면 Player 4의 힌트는 김치의 향에 대해 구체적으로 언급해 수상한 라이어로 보입니다. Player 2의 요리 재료와 조리법에 대한 설명은 자연스럽고 추측하기 어렵게 만들어, 이 플레이어가 라이어일 가능성이 있습니다. 그러나 'awdaw'라는 이상한 단어만을 남긴 Player 1의 기여도는 판단하기 어렵습니다. 전반적으로 Player 4가 김치에 대해 더 구체적이고 직접적인 언급을 했기 때문에 수상한 인물로 보입니다.

🕵️ Player 1의 의심 발언:
💡 참고 힌트:
- Player 5: 그녀가 정말 좋아하는 것은 새콤한 맛이야.
- Player 3: 발효된 맛이 나는 향긋한 반

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  ㅁㅈㄹㅈㄷㄹㅈㅁ


💬 추측: ㅁㅈㄹㅈㄷㄹㅈㅁ

🕵️ Player 4의 의심 발언:
💬 추측: Player 5: "그녀는 김치의 새콤한 맛을 좋아해, 마치 잘 익은 사과처럼." 그녀의 힌트는 제시어와 관련이 있지만 너무 직접적이지 않고 자연스럽게 들립니다.

Player 3: "발효의 향긋함, 김치가 바로 그 답이야!" 이 힌트도 논리적으로 올바르지만 조금 더 간접적입니다. Player 5의 힌트가 더 자연스러우면서도 연관성이 있습니다.

Player 1: "awdaw." 이 힌트는 이해하기 어렵고 제시어와 연결되지 않아 수상해 보입니다. 아마도 Player 1은 혼란스러워 하는 것 같군요.

Player 2: "맛있고 다재다능한 재료, 김치의 풍미로운 매력이야!" 이 문장은 지나치게 수사적이어서 의심스럽습니다. 플레이어 3의 힌트처럼 간접적으로 제시어를 참조하지만 지나치게 과장되어 있습니다.

🕵️ Player 2의 의심 발언:
💬 추측: Player 2의 언급에서 신선함과 생동감이 느껴지는데, 김치와 잘 어울릴 법한 향긋하고 발효된 맛의 반찬이 무엇인지 궁금해집니다. 하지만 Player 4는 'today's dinner'라고 언급한 것이 다소 모호해 보입니다. 그들의 발언에 더 명확성과 구체성을 제공해주세요.

🎯 ===== 라운드 2 시작 ===== 🎯
🎲 고정 힌트 발표 순서:
1. Player 5
2. Player 3
3. Player 1
4. Player 4
5. Player 2

🎙️ Player 5의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: "저녁 식사 전에 맛깔난 반찬으로, 김치 같은 것이 생각나네요."

🎙️ Player 3의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: "아, 맛있겠다! 전통을 즐겨보시는 분인지 궁금하네요."

🎙️ Player 1의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
✅ 당신은 일반 플레이어입니다. 제시어는 '김치'입니다.


...

💬 힌트: ㅁㅈㄻㅈㄻㅈ

🎙️ Player 4의 힌트:
📌 당신의 역할: 일반 플레이어
📌 주제: 음식
💬 힌트: 한국 음식에 많이 사용되곤 하는 매콤한 맛을 내는데 사용되는 발효 채소라고 할 수 있습니다.

🎙️ Player 2의 힌트:
📌 당신의 역할: 라이어
📌 주제: 음식
💬 힌트: 수프가 생각나는데, 맛도 좋고 든든해.

💭 추측 라운드 시작!

🕵️ Player 5의 의심 발언:
💬 추측: "Player 4의 힌트는 김치의 매운맛, 발효 과정 그리고 한국 음식과의 연관성을 효과적으로 전달하며 매우 자연스럽게 들립니다. 그러나 Player 3의 발언은 어제 저녁 식사를 언급하여 지나치게 구체적이고 직관적으로 제시어를 암시하고 있어 수상해 보입니다."

🕵️ Player 3의 의심 발언:
💬 추측: 음식 주제에 관해 플레이어들의 힌트들을 살펴보면, Player 5는 새콤함과 아삭함에 대한 언급으로 '김치'라는 제시어와 직접적인 연관성을 만들어내며 자연스러운 플레이어로 보입니다. 반면에 Player 1은 말이 되지 않는 글자를 사용했고, 이는 라이어를 암시할 수도 있습니다. Player 4의 설명은 발효 채소로 김치를 설명함으로써 설득력이 있어 보이지만, 매콤한 맛에 초점을 맞춘 것이 다소 모호하게 느껴져 수상할 수 있습니다. 마지막으로, Player 2는 수프와 연관 지어 언급했으나, 이는 제시어 '김치'와의 직접적인 연결고리를 제공하지 않아 약간 자연스럽지 않습니다. 따라서 논리적인 추측은:

Player 5가 김치의 새콤함과 아삭함에 대한 그들의 힌트로 보아 자연스러운 플레이어입니다. 반면 Player 1과 Player 4는 덜 직접적이거나 모호한 힌트들로 인해 라이어일 수 있습니다.

🕵️ Player 1의 의심 발언:
💡 참고 힌트:
- Player 5: "오늘 점심은 새콤하고 아삭한 맛이 정말 기분 좋게 느껴졌어요."
- Player 3: "어제 저녁에 먹은 김치볶음밥이 참 맛있었네요."
- Player 4: 한국 음식에 많이 사용되곤 하는 매

👉 누가 수상한지, 누가 아닌 것 같은지에 대해 한 마디 해주세요:  ㅁㅈㅇㅁㅈㅇㅁㅈㅇ


💬 추측: ㅁㅈㅇㅁㅈㅇㅁㅈㅇ

🕵️ Player 4의 의심 발언:
💬 추측: Player 5는 '새콤하다'와 '아삭하다' 같은 단어를 사용하며 김치의 맛을 잘 묘사해, 제시어와의 연관성이 있어 보입니다; 수상할 것 같지 않습니다. 반면에 Player 3은 김치볶음밥을 언급했지만, 그 자체만으로는 김치를 특정하지는 않아 자연스러워 보입니다. Player 1의 불명확한 문자는 추측하기 어렵게 만들고, Player 2는 수프를 언급했지만 김치에 직접적으로 관련짓지 못해 다소 수상해 보입니다. 전반적으로 이 힌트들로 볼 때, Player 3이 가장 자연스럽게 보이며 Player 2가 가장 수상해 보입니다.

🕵️ Player 2의 의심 발언:
💬 추측: Player 3의 조심스러운 어조에 주목하고 있습니다; 그들이 공유하는 정보가 너무 일반적이네요. Player 5의 설명은 흥미롭지만, 그들의 발언만으로 누군가 라이어라고 단정지기는 어렵습니다. 제시어를 모르고 있으면서도 자연스럽게 행동하는 것이 중요해요.

📊 모든 힌트와 추측 발언 요약:

📘 라운드 1 힌트:
- Player 5: 그녀가 정말 좋아하는 것은 새콤한 맛이야.
- Player 3: 발효된 맛이 나는 향긋한 반찬은 어떤 음식인가요?
- Player 1: awdaw
- Player 4: "오늘 저녁에 김치와 잘 어울리는 향긋한 음식 냄새가 났던 것 같은데, 맛있겠다는 생각이 들었어요."
- Player 2: "맛있고 다재다능한 재료로 입맛을 사로잡는 풍미 가득한 조리법이 있습니다."

📘 라운드 2 힌트:
- Player 5: "저녁 식사 전에 맛깔난 반찬으로, 김치 같은 것이 생각나네요."
- Player 3: "아, 맛있겠다! 전통을 즐겨보시는 분인지 궁금하네요."
- Player 1: ㅁㅈㄻㅈㄹ
- Player 4: "오늘 저녁, 집에서 김치와 잘 어울리는 요리를 했어."
- Player 2: "이 맛있는 요리의 재료는 다양하지만, 가장 중요한 것은 감칠맛을 내는 그 비밀 소스죠."

📘 라운드 3

Player 1의 투표 대상 (Player N 형식):  Player 3



🗳️ 최종 투표 결과:
- Player 3: 1표
- Player 1: 3표
- Player 4: 1표
- Player 5: 0표
- Player 2: 0표

💥 가장 많이 지목된 플레이어: Player 1
😈 라이어는 들키지 않았습니다! 라이어 승리!

🔒 제시어 공개: 플레이어가 라이어가 아니었다면 제시어는 아래와 같습니다.
🎯 제시어: 김치
